In [ ]:
!pip install textacy

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles ORDER BY datePublished", sql, index_col="id")

In [ ]:
import regex as re
def single_words(df, field):
    return [w for words in df[field] for w in re.split(r'\||\#', words)]

In [ ]:
from collections import Counter
from spacy.lang.de.stop_words import STOP_WORDS

nav = Counter(single_words(df, "nav"))
for w in STOP_WORDS:
    nav[w] = 0
    
voc = [w[0] for w in nav.most_common(10000)]

In [ ]:
from collections import Counter, defaultdict
from tqdm import tqdm
import regex as re

# defaultdict liefert das Argument zurück, wenn key noch unbekannt
# in diesem Fall also einen neuen (leeren) Counter
c = defaultdict(lambda: Counter())

window = 5 # sollte ungerade sein
skip = (window - 1) // 2
for doc in tqdm(df["nav"]):
    # Stopwords eliminieren
    tokens = [w for w in re.split(r'\||\#', doc) if w not in STOP_WORDS]
    for i, w in enumerate(tokens):
        if w in voc:
            for j in range(max(0, i-skip), i):
                if tokens[j] in voc:
                    c[w][tokens[j]] += 1
            for j in range(i+1, min(i+1+skip, len(tokens))):
                if tokens[j] in voc:
                    c[w][tokens[j]] += 1

In [ ]:
c["Apple"].most_common()

In [ ]:
c["Google"].most_common()

In [ ]:
import regex as re
def kwic(word, texts, window_size):
    res = []
    for text in texts:
        context = (window_size//2)*'.'
        kwic = context + r'\b' + word + r'\b' + context
        match = re.findall(kwic, text)
        for m in match:
            res.append(m)
            
    return res

In [ ]:
text = df["text"].map(str).values

In [ ]:
kwic("Apple", text, 60)